# Import
---

#### Overview
Import image data to render-ws.


In [1]:
# indirectly enable autocomplete
%config Completer.use_jedi = False

# autoreload modules
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
from itertools import chain
from natsort import natsorted
from tqdm.notebook import tqdm

from scripted_render_pipeline import basic_auth
from scripted_render_pipeline.importer import render_specs, uploader
from scripted_render_pipeline.importer.fastem_mipmapper import FASTEM_Mipmapper

#### `render-ws` environment variables

In [3]:
# render-ws environment variables
params_render = {
    "host": "https://sonic.tnw.tudelft.nl",
    "owner": "fastem",
    "project": "20230712_RF_zstack",
    "auth": basic_auth.load_auth()
}

#### Import parameters

* `dir_project` : path to project directory -- assumes subdirectories are the ROA's (which are further assumed to be sections)
* `project_path` : path to ROA directory -- assumes contents are (a grid of) tiff files
* `parallel`: how many threads to use in parallel, optimises io usage
* `clobber`: wether to allow overwriting of existing mipmaps
* `mipmap_path`: where to save mipmaps, defaults to `{project_path}/_mipmaps`

In [4]:
# set project directory
dir_project = pathlib.Path("/long_term_storage/asm_storage/asm_service/2023-07-12/20230712_RF_zstack/")

# set parameters for importing
params_import = {
    "parallel": 40,
    "clobber": True
}

# collect filepaths to ROA's
fps_ROA = natsorted([fp for fp in dir_project.glob("*") if fp.is_dir()])
fps_ROA[:5]

[PosixPath('/long_term_storage/asm_storage/asm_service/2023-07-12/20230712_RF_zstack/RF_brain_section_2_10us'),
 PosixPath('/long_term_storage/asm_storage/asm_service/2023-07-12/20230712_RF_zstack/RF_brain_section_3_10us'),
 PosixPath('/long_term_storage/asm_storage/asm_service/2023-07-12/20230712_RF_zstack/RF_brain_section_4_10us'),
 PosixPath('/long_term_storage/asm_storage/asm_service/2023-07-12/20230712_RF_zstack/RF_brain_section_5_10us'),
 PosixPath('/long_term_storage/asm_storage/asm_service/2023-07-12/20230712_RF_zstack/RF_brain_section_6_10us')]

#### Create mipmaps

In [5]:
# loop through ROA's
layers = []
for z, fp in tqdm(
    enumerate(fps_ROA),
    total=len(fps_ROA)
):

    # update directory with each ROA / section
    params_import["project_path"] = fp / "corrected"

    # make mipmaps
    mipmapper = FASTEM_Mipmapper(**params_import)
    layer = mipmapper.create_all_mipmaps()[0]
    layers.append(layer)

  0%|          | 0/17 [00:00<?, ?it/s]


making mipmaps: 100%|███████████████████████████████████████████████████████████████████| 9/9 [00:04<00:00,  1.91img/s]

making mipmaps: 100%|███████████████████████████████████████████████████████████████████| 9/9 [00:04<00:00,  1.87img/s]

making mipmaps: 100%|███████████████████████████████████████████████████████████████████| 9/9 [00:04<00:00,  1.90img/s]

making mipmaps: 100%|███████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.62img/s]

making mipmaps: 100%|███████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.76img/s]

making mipmaps: 100%|███████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.74img/s]

making mipmaps: 100%|███████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.52img/s]

making mipmaps: 100%|███████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.71img/s]

making mipmaps: 100%|██████████

#### Import to render-ws

In [6]:
# initialize uploader
uppity = uploader.Uploader(
    **params_render,
    clobber=params_import.get("clobber")
)

# create (Tibbe) stack
stack = render_specs.Stack(
    name="corrected",
    specs=list(chain(*[layer.tilespecs for layer in layers])),
    pixel_size=layer.pixel_size
)

# import stack to render-ws
uppity.upload_to_render(
    stacks=[stack],
    z_resolution=100
)

uploading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.22stacks/s]


#### Plot stack

In [7]:
import sys

dir_code = "../src/"
if dir_code not in sys.path:
    sys.path.insert(1, dir_code)

import plotting

In [8]:
# plot stack
plotting.plot_stacks(
    stacks=[stack.name],
    render=uppity.render
)

  0%|          | 0/1 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=16), IntSlider(value=30207, description='vmin', …